In [3]:
import pandas as pd
import numpy as np
import csv

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [4]:
out_file_int = "data/data_int.txt"
list_int = [1,2,3,4,5,6]
with open(out_file_int, 'a') as outfile:
    outfile.write(str(list_int)) 
    
#!cat data/data_int.txt  --> This is the command cat for unix users
!type data\"data_int.txt" 

    
out_file_float = "data/data_float.txt"
#I create a random matrix of float from 0.5 to 10
matrix_float = np.random.uniform(low=0.5, high=10, size=25).reshape((5,5))
with open(out_file_float, 'a') as outfile:
    outfile.write(str(matrix_float)) 

!type data\"data_float.txt" 
    
read_file = pd.read_csv(r'data\data_float.txt')
read_file.to_csv(r'data\data_float.csv', index=None)

[1, 2, 3, 4, 5, 6]
[[3.42305237 5.98496046 1.07966151 8.91752691 7.5890655 ]
 [0.97008928 7.07601174 9.05980613 5.46591452 9.56025857]
 [9.47223551 8.15475296 6.48385518 7.50267588 6.92022329]
 [8.04267382 8.46506342 0.99585437 7.23975891 8.99255192]
 [9.71062833 6.05600023 5.38014667 6.54802134 1.01976314]]


2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [16]:
import urllib.request
import json 
import requests

#------The following code is not the correct answare, I need to find a way to read the file directlu with the url
file = 'data/user_data.json'
f = open(file)
data = json.load(f)
#------
        
filtered_data = [x for x in data if (x['CreditCardType'] == 'American Express')]
read_file = pd.DataFrame(filtered_data)
read_file.to_csv (r'data\filtered_CSV.csv', index=None)

3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [17]:
#url_csv = "https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv"

#------The following code is not the correct answare, I need to find a way to read the file directlu with the url
df = pd.read_csv("data/mushrooms_categorized.csv")
#------


gr_class = df.groupby('class').mean()
read_file.to_json (r'data\Average_feature_JSON.json')

4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [38]:
dat_file = r"data\credit_card.dat"   

count = 1
CHUNK_SIZE = 6
with open(dat_file) as f:
    chunk = f.read(CHUNK_SIZE)
    while chunk:
        if (count == 5):
            CHUNK_SIZE = 7
            count = 1
            print("White space")
        else:
            CHUNK_SIZE = 6
            num = int(chunk, 2)
            count += 1
            print(num)
        chunk = f.read(CHUNK_SIZE)

55
54
52
56
White space
107
45
47
39
White space
79
31
31
22
White space
22
22
62
13
White space
121
26
59
48
White space
50
52
55
32
White space
103
43
41
1
White space
75
27
26


ValueError: invalid literal for int() with base 2: '10\n110'

5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3